In [ ]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from scipy.stats import norm
from copy import deepcopy

In [ ]:
import os
import logging
import numpy as np
from datetime import datetime

## Pre-Processing

### Add Virtual starting node

In [ ]:
def find_add(graph):   #this function finds and adds a new virtual entry node only if there are mult. entry nodes present 
    
    # collect nodes with no incoming edges
    roots = []
    for node, indeg in graph.in_degree():
        if indeg == 0:
           roots.append(node)

    l = len(roots) #length

    # create single entry node
    if l > 1:
        entry_node = 0
        graph.add_node(entry_node)

        # link entry node to each root
        for r in roots:
            graph.add_edge(entry_node, r, weight=DEFAULT_WEIGHT_VALUE)

        return entry_node, graph, roots
    else:
       # only one root, that's the entry node
       return roots[0], graph, roots

### Add Virtual Target Node 

In [ ]:
def merge_targets(origgraph):  # this function merges the targets
    
    # identifying nodes with no outgoing edges
    targets = []
    for node, outdegree in origgraph.out_degree():
        if outdegree == 0:
            targets.append(node)

    # count how many targets
    l = len(targets)

    # if there are only 0 or 1 targets, no need to merge
    if l <= 1:
        return origgraph

    # convert target nodes into string labels
    tstrings = []
    for t in targets:
        tstrings.append(str(t))

    # join target strings into merged label
    merged_label = "c("
    for i in range(len(tstrings)):

        merged_label += tstrings[i]
        if i != len(tstrings) - 1:
            merged_label += ","
    merged_label += ")"

    # create a new graph
    newgraph = nx.MultiDiGraph()

    # collect non-target nodes
    non_targets = []
    for node in origgraph.nodes():
        if node not in targets:
            non_targets.append(node)

    # add all non-target nodes to new graph
    for n in non_targets:
        newgraph.add_node(n)

    # add the merged virtual target node
    newgraph.add_node(merged_label)

    # track edges that originally pointed to targets
    pred_targetedges = {}
    for u, v, data in origgraph.edges(data=True):
        if v in targets:
            if u not in pred_targetedges:
                pred_targetedges[u] = []
            # explicitly handle default weight
            if 'weight' in data:
                w = data['weight']
            else:
                w = DEFAULT_WEIGHT_VALUE
                
            pred_targetedges[u].append((w, v))

    # recreate edges to the virtual target
    for u, edges in pred_targetedges.items():
        # count number of occurrences for each weight
        wcounts = {}
        for weight, _ in edges:
            if weight not in wcounts:
                wcounts[weight] = 1
            else:
                wcounts[weight] += 1

        # add edges according to counts
        for weight, count in wcounts.items():
            for _ in range(count):
                newgraph.add_edge(u, merged_label, weight=weight)

    # copy over all other edges not involving target nodes
    for u, v, data in origgraph.edges(data=True):
        if u not in targets and v not in targets:
            edge_attr = {}
            for key in data:
                edge_attr[key] = data[key]
            newgraph.add_edge(u, v, **edge_attr)

    # return the new graph
    return newgraph

## Elements preparation for the Game

In [ ]:
def generate_game_elements(graph, entry_node, original_roots):
    # initial vars
    g=graph; startNode=entry_node 
    routes=[]; V=[]; as1=[]; as2=[]; adv_list=[] # declare the vars I am going to use
    
    # 1. create a list with all targets in G 
    # loop
    tg_list = []
    nodes = list(g.nodes())
    i = 0
    while i < len(nodes):
        n = nodes[i]
        # check out degree
        if g.out_degree(n) == 0:
            tg_list.append(n)
        i=i+1

    if len(tg_list) != 1:
        print("Warn: target count " + str(len(tg_list)))

    # 2. create a list of all unique paths (for atk) and store them in routes
    # raw = nx.all_simple_paths(g, startNode, tg_list[0])
    gen = nx.all_simple_paths(g, startNode, tg_list[0])
    raw = list(gen)
    
    # manual deduplication
    seen = []
    j = 0
    while j < len(raw):
        p = raw[j]
        k = tuple(p)
        
        # check existence
        found = False
        k_idx = 0
        while k_idx < len(seen):
            if seen[k_idx] == k:
                found = True
                break
            k_idx+=1
            
        if found == False:
            seen.append(k); routes.append(p)
        j=j+1

    # 3. get V
    tmp_v = []
    x = 0
    while x < len(routes):
        path = routes[x]
        y = 0
        while y < len(path):
            tmp_v.append(path[y])
            y+=1
        x+=1

    # All nodes in V have to be unique
    z = 0
    while z < len(tmp_v):
        curr = tmp_v[z]
        # if curr not in V:
        in_v = False
        v_idx = 0
        while v_idx < len(V):
            if V[v_idx] == curr:
                in_v = True
            v_idx+=1
            
        if not in_v:
            V.append(curr)
        z=z+1
            
    V.sort(key=str)

    # topo sort 
    full_topo = list(nx.topological_sort(g))
    node_order = []
    
    t_i = 0
    while t_i < len(full_topo):
        node = full_topo[t_i]
        if node in V:
            node_order.append(node)
        t_i += 1

    # 4. Lists
    # create as1 such that it does not include starting nodes, target nodes, root nodes
    nonDef_nodes = []
    nonDef_nodes.append(startNode)
    for t in tg_list: nonDef_nodes.append(t)
    for r in original_roots: nonDef_nodes.append(r)
    
    ai = 0
    while ai < len(V):
        n = V[ai]
        # check bad
        is_bad = False
        for b in nonDef_nodes:
            if n==b: is_bad=True
            
        if is_bad==False:
            as1.append(n)
        ai+=1
    
    # create as2 for atk strategies
    as2 = routes

    # 5. adv
    # adv_list declared at top
    Adv_no_go = [startNode]
    for t in tg_list: Adv_no_go.append(t)
    
    vi = 0
    while vi < len(V):
        vn = V[vi]
        if vn not in Adv_no_go:
            adv_list.append(vn)
        vi+=1
            
    if len(adv_list)==0:
        print("debug: adv list empty")

    theta = {}
    if len(adv_list) > 0:
        cnt = float(len(adv_list))
        val = 1.0/cnt
        
        for a in adv_list:
            theta[a] = val

    m = len(routes)

    return routes, V, as1, as2, tg_list, node_order, adv_list, theta, m

## Calculate Payoffs based on the game elements

This method is mostly useless and can be removed long term. In my current implementation all we care about is the last entry of U = [0.17, 0.10, 0.63, 0.10, 1e-7] which represents the prob. of the attacker to reach the target node.

And per one U (Node Check & Attack Path pair), we extract only this one value for the final pay off matrix. 
lossDistribution() does not add anything of value.

In [ ]:
# Note: The current version is completely useless. 
# I wrote it here because the R version had it, but as of now it does not do anything
# What I need is really just the last entry of U
# But this method does not change U in any way
# might be usefule some day for some added complexity ? Leaving it in for now...
def lossDistribution(U):
    return {
        'dpdf': U,  
        'support': np.arange(1, len(U) + 1),
        'cdf': np.cumsum(U),
        'tail': 1 - np.cumsum(U) + U,
        'range': [1, len(U)]
    }

#### Calculate Payoffs for each path/check pair

In [ ]:
def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                  attack_rate, defense_rate, node_order):
    # init vars
    payoffs = []; i = 0; 

    #print(payoffs)
    
    # 1. process checks
    while i < len(as1):
        check = as1[i]
        
        # loop over paths (as2)
        j = 0
        while j < len(as2):
            path = as2[j]
            
            # init U vector
            U = np.zeros(len(V))
            
            # 2. iterate through all avatars
            k = 0
            while k < len(adv_list):
                avatar = adv_list[k]
                L = np.zeros(len(V))
                
                # check if avatar in path
                in_path = False
                start_idx = -1
                
                p_idx = 0
                while p_idx < len(path):
                    if path[p_idx] == avatar:
                        in_path = True
                        start_idx = p_idx
                        break
                    p_idx = p_idx + 1
                
                # 2a. on path logic
                if in_path == True:
                    # slice path 
                    route = []
                    r_idx = start_idx
                    while r_idx < len(path):
                        route.append(path[r_idx])
                        r_idx = r_idx + 1
                        
                    # get probs
                    pdf_d = random_steps_fn(route, attack_rate, defense_rate, graph)
                    
                    # 2b. check point adjustment
                    # check if check node is in route
                    cutPoint = len(route) # default
                    c_idx = 0
                    while c_idx < len(route):
                        if route[c_idx] == check:
                            cutPoint = c_idx + 1
                            break
                        c_idx = c_idx + 1
                        
                    # truncating pdf
                    pdf_subset = pdf_d[:cutPoint]
                    s = np.sum(pdf_subset)
                    
                    payoffDistr = []
                    if s < 1e-15:
                        payoffDistr = np.zeros(cutPoint)
                        payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_subset / s
                        
                    # map to L vector
                    route_subset = route[:cutPoint]
                    
                    # map values 
                    r_s_idx = 0
                    while r_s_idx < len(route_subset):
                        node = route_subset[r_s_idx]
                        val = payoffDistr[r_s_idx]
                        
                        # find index in V
                        v_idx = -1
                        v_search = 0
                        while v_search < len(V):
                            if V[v_search] == node:
                                v_idx = v_search
                                break
                            v_search = v_search + 1
                            
                        if v_idx != -1:
                            L[v_idx] = val
                        r_s_idx = r_s_idx + 1
                        
                # 2c. off path logic
                else:
                    # find avatar index in V
                    idxOfNode = -1
                    z = 0
                    while z < len(V):
                        if V[z] == avatar:
                            idxOfNode = z
                            break
                        z = z + 1
                        
                    if idxOfNode != -1:
                        L[idxOfNode] = 1.0

                # 2d. weight by theta
                th = theta[avatar]
                U = U + (th * L) 
                
                k = k + 1
            
            # 3. cleanup U
            u_sum = np.sum(U)
            if u_sum < 1e-15:
                U = np.full_like(U, 1e-7)
            else:
                U = U / u_sum
                # replace small values
                U = np.where(U < 1e-7, 1e-7, U)
                
            # reorder based on node_order
            #  reorder
            new_U = np.zeros(len(U))
            
            ni = 0
            while ni < len(node_order):
                n = node_order[ni]
                # find old index in V
                old_idx = -1
                vi = 0
                while vi < len(V):
                    if V[vi] == n:
                        old_idx = vi
                        break
                    vi += 1
                
                if old_idx != -1:
                    new_U[ni] = U[old_idx]
                ni += 1
                
            U = new_U

            # 4. smooth/loss
            ld = lossDistribution(U)
            payoffs.append(ld)
            
            j = j + 1
        i = i + 1

    return payoffs

## Find the Equilibrium

In [ ]:
def solve_game(payoffs, as1, as2):

    # initialize payoff matrix
    payoff_mat = np.zeros((len(as1), len(as2)))
    for i in range(len(as1)):
        for j in range(len(as2)):
            idx = i * len(as2) + j
            ld = payoffs[idx]
            lv = ld['dpdf'][-1]  # taking last prob
            payoff_mat[i, j] = lv

    # debug print for surity
    if debug_mode:
        print("\nDebug: checking payoff matrix")
        print("Dimensions: %d x %d" % (len(as1), len(as2)))
        for i in range(len(as1)):
            row_str = "row %d:" % (i + 1)
            for j in range(len(as2)):
                val = payoff_mat[i, j]
                row_str += " %.6f" % val
            print(row_str)
        print("End debug\n")

    # setup LP for defender
    c = np.zeros(len(as1) + 1)
    c[0] = 1.0  # constraints defender

    # A_ub the payoff matrix plus one special column with -1 entries for w (look in your goodnotes notes)
    A_ub = np.zeros((len(as2), len(as1) + 1))
    # b_up represents the zero vector, right hand size of the inequality and is filled with 0s
    b_ub = np.zeros(len(as2))


    for j in range(len(as2)):
        # the first col will be filled with the -1 entries for w
        A_ub[j, 0] = -1.0
        # fills up the rest with payoff entries 
        for i in range(len(as1)):
            A_ub[j, i + 1] = payoff_mat[i, j]

            
    # equality constraint sets first value to 0, rest to 1s 
    A_eq = np.zeros((1, len(as1) + 1))
    for k in range(1, len(as1) + 1):
        A_eq[0, k] = 1.0  # set each prob. coeff. to 1
    b_eq = np.array([1.0])

    # defines none are negative
    bounds = [(0, None)] * (len(as1) + 1)

    # solve the LP for the defender
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

    # in case no defender solution
    if not result.success:
        logger.info("Defender LP failed, stop early...")
        return None


    def_successprob = result.x[0]
    def_probs = result.x[1:]
    temp_def = def_probs.copy()


    # setup LP for attacker 
    c_att = np.zeros(len(as2) + 1)
    c_att[0] = -1.0  # constraints attacker

    # almost identical constraints set up as before, we just need to make sure that the 
    # inequality sign is flipped this time
    A_ub_att = np.zeros((len(as1), len(as2) + 1))
    b_ub_att = np.zeros(len(as1))
    for i in range(len(as1)):
        A_ub_att[i, 0] = 1.0
        for j in range(len(as2)):
            A_ub_att[i, j + 1] = -payoff_mat[i, j]

    # equality constraints
    A_eq_att = np.zeros((1, len(as2) + 1))
    for k in range(1, len(as2) + 1):
        A_eq_att[0, k] = 1.0
    b_eq_att = np.array([1.0])
    bounds_att = [(0, None)] * (len(as2) + 1)

    # solve the LP for the attacker
    res_att = linprog(c_att, A_ub=A_ub_att, b_ub=b_ub_att, A_eq=A_eq_att, b_eq=b_eq_att, bounds=bounds_att)

    # in case attacker LP does not give me any results 
    if not res_att.success:
        logger.info("Attacker LP failed")
        return None

    # extract results for later storage
    att_probs = res_att.x[1:]
    att_succprob = res_att.x[0]
   
    # map defender
    defender_strategymap = {}
    for k, v in zip(as1, temp_def):
        defender_strategymap[k] = v  # store mapping

    # return results
    return {
        "optimal_defense": defender_strategymap,
        "attacker_strategy": att_probs,
        "defender_success": def_successprob,
        "attacker_success": att_succprob
    }


### Method to run the Actual Game

In [ ]:
def run_game(attacker_graph, defender_graph=None, attack_rate_list=None, dropped=None, defense_rate_list=None, random_steps_fn=None):

    # if no defender graph, use attacker one
    if defender_graph is None: 
        defender_graph = attacker_graph  

    final_eq = None 

    # ----- attacker prep -----
    if debug_mode: print_debug_info(attacker_graph, "This is the Attacker Graph (original)")

    atkEntry, attGraph, atkRoots = find_add(attacker_graph)
    attGraph = merge_targets(attGraph)  # merge mult. targets and take care of mult. edges

    if debug_mode: 
        print_debug_info(attGraph, "After merging targets of attack graph")

    # ----- attacker elements -----
    _, V, _, atkPaths, tgtList, nodeOrder, advList, theta, _ = generate_game_elements(
        attGraph, atkEntry, atkRoots
    )
    va = None  

    # ----- defender prep -----
    if debug_mode: print_debug_info(defender_graph, "This is the Defender Graph (original)")

    defEntry, defGraph, defRoots = find_add(defender_graph)
    defGraph = merge_targets(defGraph)

    if debug_mode: print_debug_info(defGraph, "After merging targets of the Defender Graph")

    # ----- defender elements --
    _, _, defSpots, _, _, _, _, _, _ = generate_game_elements(defGraph, defEntry, defRoots)
    # return routes, V, as1, as2, tg_list, node_order, adv_list, theta, m
    # log attack paths
    logger.info("\nDebug - Current Attack paths:")
    for i,p in enumerate(atkPaths): 
        logger.info(f"  {i}: {p}")  # original print

    # debug dropped nodes
    if debug_mode:
        print("\n Dropped nodes:", dropped)
        debug_paths(defSpots, atkPaths)

    # defaults
    attack_rate_list = attack_rate_list or [0]
    defense_rate_list = defense_rate_list or [0]

    # loop over rates
    for dr in defense_rate_list:
        for ar in attack_rate_list:

            logger.info("\n+++++++++++++++++++++++++++++")
            logger.info(f"\nThe virtual target nodeid is {tgtList[0]}\n")
            logger.info(f"attack rate =  {ar} , defense rate =  {dr} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")

            # calc payoff
            payoffs = calculate_payoff_distribution(
                attGraph, defSpots, atkPaths, V, advList, theta, random_steps_fn, ar, dr, nodeOrder
            )

            eq = solve_game(payoffs, defSpots, atkPaths)
            if eq is None: continue

            final_eq = eq

            # --- log optimal defense ---
            logger.info("optimal defense strategy:")
            logger.info("         prob.")
            for node, prob in sorted(eq['optimal_defense'].items(), key=lambda x:str(x[0])):
                logger.info(f"{node} {prob:.6e}")

            # --- log worst-case attacker ---
            logger.info("\nworst case attack strategies per goal:")
            logger.info("          1")
            if 'attacker_strategy' in eq:
                for idx, prob in enumerate(eq['attacker_strategy'], 1):
                    logger.info(f"{idx} {prob:.7f}")

            logger.info(f"[1] {eq['attacker_success']:.3f}")
            logger.info(f"\nDefender can keep attacker success below: {eq['defender_success']:.3f}")
            logger.info(f"Attacker can guarantee success probability of: {eq['attacker_success']:.3f}")

    return final_eq

### Method to be called in different Notebook

In [ ]:
def main():
    # setup variables
    baseline_result = None
    
    i = 0; diff = 0.0; sign = "" # mixed init
    
    # Make sure we're using the main logger
    switch_logger(use_subgraph_logger=False)
    
    logger.info("\n\n")
    logger.info("="*80)
    logger.info("BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE")
    logger.info("="*80)
    logger.info("\n")
    
    # Run the baseline
    print("We start with the baseline graph calculation!")
    baseline_result = run_game(attacker_graph=full_attack_graph, defender_graph=full_attack_graph, attack_rate_list=attack_rate_list, 
                             defense_rate_list=defense_rate_list, random_steps_fn=random_steps)
    
    # If toggle is false, also run subgraph analysisb for 0day exploits
    print("Now we are going to run the subgraph analysis")
    
    # check
    if RUN_BASELINE_ONLY == False:
        # Switch to subgraph logger
        switch_logger(use_subgraph_logger=True)
        
        logger.info("\n")
        logger.info("="*80)
        # manual concat for logger
        msg = "STARTING SUBGRAPH ANALYSIS WITH " + str(len(defender_subgraphs_list)) + " DEFENDER SUBGRAPHS"
        logger.info(msg)
        logger.info("Drop percentage is : " + str(current_drop_percentage))
        logger.info("="*80)
        logger.info("\n")
        
        # clear lists
        attacker_success_values = []
        total_nodes_dropped_list = []
        
        # Running the subgraph analysis
        # i = 0 (already declared)
        while i < len(defender_subgraphs_list):
            #  unpack
            curr = defender_subgraphs_list[i]
            defender_subgraph = curr[0]
            dropped_nodes = curr[1]
            cleaned_nodes = curr[2]

            # Tracking total nodes dropped 
            c1 = 0
            for x in dropped_nodes: c1 = c1 + 1
            c2 = 0
            for y in cleaned_nodes: c2 = c2 + 1
            
            total_nodes_dropped = c1 + c2
            total_nodes_dropped_list.append(total_nodes_dropped)

            logger.info("\n")
            logger.info("-"*60)
            logger.info("SUBGRAPH RUN #" + str(i+1) + ": DEFENDER HAS LIMITED NETWORK VISIBILITY")
            logger.info("Drop percentage is: " + str(current_drop_percentage))
            
            #map convert
            d_str = ', '.join(map(str, dropped_nodes))
            logger.info("Node(s) Nr. " + d_str + " were dropped from this graph")
            logger.info("Additionally " + str(len(cleaned_nodes)) + " nodes were cleaned afterwards")
            logger.info("So in total " + str(total_nodes_dropped) + " Nodes were dropped")
            logger.info("-"*60)
            logger.info("\n")

            # Prepare the graph
            defender_subgraph_current = deepcopy(defender_subgraph)

            result = run_game(attacker_graph=full_attack_graph, defender_graph=defender_subgraph_current, dropped = dropped_nodes,
                            attack_rate_list=attack_rate_list, 
                            defense_rate_list=defense_rate_list, random_steps_fn=random_steps)
            
            #  dict check
            if result != None:
                # check key 
                found_key = False
                k_list = list(result.keys())
                for k in k_list:
                    if k == 'attacker_success':
                        found_key = True
                
                if found_key == True:
                    val = result['attacker_success']
                    attacker_success_values.append(val)
            
            i = i + 1

        # Calculating and logging the average
        if len(attacker_success_values) > 0:
            # manual average calculation
            total_acc = 0.0
            idx = 0
            while idx < len(attacker_success_values):
                total_acc = total_acc + attacker_success_values[idx]
                idx = idx + 1
            avg_attacker_success = total_acc / float(len(attacker_success_values))
            
            # average dropped
            total_drop = 0
            idx2 = 0
            while idx2 < len(total_nodes_dropped_list):
                total_drop = total_drop + total_nodes_dropped_list[idx2]
                idx2 = idx2 + 1
            avg_total_nodes_dropped = total_drop / float(len(total_nodes_dropped_list))

            # Getting total number of nodes
            total_nodes = full_attack_graph.number_of_nodes()

            # Calculating percentages
            # init_drop = (len(dropped_nodes) / total_nodes) * 100
            init_drop = (float(len(dropped_nodes)) / float(total_nodes)) * 100.0
            tot_drop_pct = (avg_total_nodes_dropped / float(total_nodes)) * 100.0

            logger.info("\n\n")
            logger.info("="*80)
            logger.info("SUBGRAPH ANALYSIS SUMMARY")
            logger.info("="*80)
            logger.info("Number of subgraphs analyzed: " + str(len(attacker_success_values)))
            logger.info("Total number of nodes in the graph: " + str(total_nodes))
            logger.info("Initial number of dropped nodes per subgraph: " + str(len(dropped_nodes)))
            
            # formatting 
            logger.info("% of dropped nodes (initially): " + "{:.1f}".format(init_drop) + "%")
            logger.info("Average total nodes dropped (including cleanup): " + "{:.2f}".format(avg_total_nodes_dropped))
            logger.info("% of total nodes dropped (including cleanup): " + "{:.1f}".format(tot_drop_pct) + "%")
            
            base_val = baseline_result['attacker_success']
            logger.info("Baseline attacker success: " + "{:.3f}".format(base_val))
            logger.info("Average attacker success across subgraphs: " + "{:.3f}".format(avg_attacker_success))
            
            # calc diff 
            diff = avg_attacker_success - base_val
            sign = ""
            if diff > 0:
                sign = "+"
            
            # print("DEBUG: diff is " + str(diff)) 
            logger.info("Difference from baseline: " + sign + "{:.3f}".format(diff))
            logger.info("="*80)
            print("The subgraph analysis is done!")